# Import the needed packages 

In [1]:
import erroranalysis as err
import matplotlib.pyplot as plt
import sys
import csv
csv.field_size_limit(min(sys.maxsize, 2147483646))

2147483646

In [2]:
Z, C, W, M = err.load_data('./data/SG_data.csv')
ZRef, CRef, WRef, MRef = err.load_data('./data/SG_data_ref.csv')

In [ ]:
RMSv = err.Root_Mean_Squared_Velocity(Z, C, W, 'Meridional')

# Create a line plot
plt.plot(RMSv)

# Add labels and title (optional)
plt.xlabel('Time Step')
plt.ylabel('RMSv')
plt.title('RMSv')

# Display the plot (if running in a Jupyter Notebook, this step might not be necessary)
plt.show()

In [ ]:
euclid_error = err.Weighted_Euclidian_Error(Z, ZRef, MRef, 5, 3)

print(euclid_error)

In [3]:
sinkhorn_loss = err.Sqrt_Sinkhorn_Loss(Z, M, ZRef, MRef, 0.001, 5, 2.5)

[-6.171028e-10 -6.171028e-10 -6.171028e-10 ... -6.394546e-10 -3.526076e-10
 -3.898587e-10] [-0.00543556 -0.00543556 -0.00543556 ... -0.00543556 -0.00543557
 -0.00543553]
[ 1.4386729e-08 -4.8770994e-11 -4.8770994e-11 ... -9.3474568e-11
 -8.2298675e-11  1.7170201e-07] [-0.00543667 -0.00543559 -0.00543557 ... -0.00543556 -0.00543556
 -0.00544487]


In [4]:
print(sinkhorn_loss)

0.33819934028100673
